In [1]:
"""
Author: Elisha-Wigwe Chijioke O.
Project Name: Artificial Neural Networks - Classification (Diabetes)
_Building a neural network from scratch (with NumPy)_
"""

import numpy as np
from pdb import set_trace

In [8]:
from numpy import reshape, array, zeros, random, exp, ones

class Network(object):
    def __init__ (self, sizes):
        random.seed(1)
        self.sizes = sizes
        self.num_of_layers = len(sizes)
        self.size = self.num_of_layers - 1
        self.no_features = self.sizes[0]		
        self.weights = [2 * random.random((a, b)) - 1 for a, b in zip(sizes[:-1], sizes[1:]) ]
        self.biases = [2 * random.random((a, 1)) - 1 for a in sizes[1:]]		
        self.weight_deltas = [ zeros(w.shape) for w in self.weights]
        self.bias_deltas = [ zeros(b.shape) for b in self.biases]
        self.bias_velocities = [ zeros(b.shape) for b in self.biases]
        self.weight_velocities = [ zeros(w.shape) for w in self.weights]

    def evaluate (self, test_data, accuracy=0.0):
        total = len(list(test_data))
        test_data = [ ( self.normalize(xt), yt) for xt, yt in test_data ]
        for x_test, y_test in test_data:
            act = reshape(x_test, (self.no_features, 1))
            for l in range(self.size):
                act = self.sigmoid(self.weights[l].T.dot(act) + self.biases[l])
            if self.is_correct(act, y_test):
                accuracy += 1
        return accuracy, total


    def is_correct (self, act, target):
        return np.round(act) == target

    
    def predict(self, data):
        for l in range(self.size):
            data = self.sigmoid(self.weights[l].T.dot(data) + self.biases[l])
            return data

    # This is the Min-Max Normalization
    def normalize (self, xtrain):
        return (xtrain - xtrain.min()) / (xtrain.max() - xtrain.min())

    def standardize (self, xtrain):
        return (xtrain - xtrain.mean()) / xtrain.std()

    def train (self, training_data, epochs=10000000, momentum_factor=0.5, check=10, lr=0.03, mini_batch_size=10, test_data=None):
        train_list = list(training_data)
        test_list = list(test_data)
        
        total = len(train_list)
        
        train_list = [ ( self.normalize(xt), yt) for xt, yt in train_list ]
        
        if mini_batch_size == None: mini_batch_size = total
        random.shuffle(train_list)
        
        mini_batches = [ train_list[k:k+mini_batch_size] for k in range(0, total, mini_batch_size)]
        
        for iter in range(epochs):
            for mini_batch in mini_batches:
                batch_total = len(mini_batch)
                for x, y in mini_batch:
                    activation = reshape(x, (self.no_features, 1))
                    activations = [activation]
                    for we, bi in zip(self.weights, self.biases):
                        activation = self.sigmoid(we.T.dot(activation) + bi)
                        activations.append(activation)

                    error_derivative = self.cost_function_derivative(activations[-1], y)
                    delta = error_derivative * self.sigmoid_derivative(activations[-1])
                    
                    for l in range(self.size):
                        self.bias_deltas[-l-1] = delta
                        self.weight_deltas[-l-1] = activations[-l-2].dot(delta.T)
                        delta = (self.weights[-l-1] * self.sigmoid_derivative(activations[-l-2])).dot(delta)

                    for l in range(self.size):
                        self.bias_velocities[l] = (momentum_factor * self.bias_velocities[l]) - ((lr/batch_total) * self.bias_deltas[l])
                        self.weight_velocities[l] = (momentum_factor * self.weight_velocities[l]) - ((lr/batch_total) * self.weight_deltas[l])
                        self.biases[l] = self.biases[l] + self.bias_velocities[l]
                        self.weights[l] = self.weights[l] + self.weight_velocities[l]

            if iter % check == 0:
                accuracy = 0.0
                if test_data != None:
                    accuracy, test_total = self.evaluate(test_list)
                else:
                    accuracy, test_total = self.evaluate(train_list)

                print("Epoch: {}, Accuracy: {:.4f}%".format(iter, 100*(accuracy/test_total)))

        return (self.weights, self.biases)

    def sigmoid (self, x):
        return 1 / (1 + exp(-x))

    def sigmoid_derivative (self, x):
        return x * (1 - x)

    def cost_function_derivative (self, output, target):
        # This is the derivative of the Mean-Square Error Cost Function
        return output - target

In [9]:
pima_data = np.genfromtxt('./pima.csv', delimiter=',')

split_factor = 0.7
split = int(split_factor*len(pima_data))

train_data = pima_data[:split]
test_data = pima_data[split:]

num_of_features = 8

x_train, x_test = train_data[:, 0:num_of_features], test_data[:, 0:num_of_features]
y_train, y_test = train_data[:, [num_of_features]], test_data[:, [num_of_features]]

training_data = zip(x_train, y_train)
test_data = zip(x_test, y_test)

# instantiate the network
net = Network([num_of_features, 500, 500, 1])
lr = 0.03



In [ ]:
net.train(training_data, epochs=1000, momentum_factor=0.3, mini_batch_size=10, lr=0.005, check=10, test_data=test_data)

Epoch: 0, Accuracy: 56.7100%
Epoch: 10, Accuracy: 57.1429%
Epoch: 20, Accuracy: 58.0087%
Epoch: 30, Accuracy: 59.3074%
Epoch: 40, Accuracy: 59.7403%
Epoch: 50, Accuracy: 60.6061%
Epoch: 60, Accuracy: 61.0390%
Epoch: 70, Accuracy: 61.4719%
Epoch: 80, Accuracy: 61.4719%
Epoch: 90, Accuracy: 61.4719%
Epoch: 100, Accuracy: 61.4719%
Epoch: 110, Accuracy: 61.0390%
Epoch: 120, Accuracy: 61.9048%
Epoch: 130, Accuracy: 61.0390%
Epoch: 140, Accuracy: 61.0390%
Epoch: 150, Accuracy: 60.6061%
Epoch: 160, Accuracy: 60.6061%
Epoch: 170, Accuracy: 60.6061%
Epoch: 180, Accuracy: 60.6061%
Epoch: 190, Accuracy: 60.1732%
Epoch: 200, Accuracy: 60.6061%
Epoch: 210, Accuracy: 61.4719%
Epoch: 220, Accuracy: 61.4719%
Epoch: 230, Accuracy: 62.3377%
Epoch: 240, Accuracy: 62.3377%
Epoch: 250, Accuracy: 62.3377%
Epoch: 260, Accuracy: 61.9048%
Epoch: 270, Accuracy: 62.7706%
Epoch: 280, Accuracy: 62.7706%
Epoch: 290, Accuracy: 62.7706%
Epoch: 300, Accuracy: 62.7706%
Epoch: 310, Accuracy: 62.7706%
Epoch: 320, Accurac